## Dashboard for displaying visualizations

In [29]:
import numpy as np
import pandas as pd
import altair as alt
from dash import dash, dcc, html, Input, Output
from altair import datum

In [34]:
nav_events = pd.read_csv('hackathon_alternate_data/navigation_events.csv')
disc = pd.read_csv('hackathon_alternate_data/additional/discussions.csv')
disc_dates = pd.read_csv('hackathon_alternate_data/additional/discussion_topics.csv')

In [38]:
disc['timestamp'] = pd.to_datetime(disc['timestamp'])
disc['date'] = disc['timestamp'].dt.normalize()#date
disc_merged = disc.merge(disc_dates[['title','todo_date_date']],how='inner',left_on='discussion_topic_title',right_on='title')

In [53]:
# Plot hist of assignment submission dates
def plot_altair(assignment):
    chart = alt.Chart(disc_merged).mark_bar().encode(
        alt.X('date:T').title("Date"),
        alt.Y('count(x):N').title("Number of Submissions")).transform_filter(datum.discussion_topic_title == assignment)
    line = chart.mark_rule(color='red').encode(x = 'todo_date_date:T')
    return (chart+line).to_html()

assignments = disc['discussion_topic_title'].unique()

# Set up dashboard
app = dash.Dash(__name__, external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])

app.layout = html.Div([
        dcc.Dropdown(
            id='assignment', value='Casual Area',
            options=[{'label': i, 'value': i} for i in assignments]),
        html.Iframe(
            id='scatter',
            style={'border-width': '0', 'width': '100%', 'height': '400px'},
            srcDoc=plot_altair(assignment='Casual Area'))])

@app.callback(
    Output('scatter', 'srcDoc'),
    Input('assignment', 'value'))
def update_output(assignments):
    return plot_altair(assignments)

if __name__ == '__main__':
    app.run_server(debug=True) 